In [19]:
!pip install pandas numpy matplotlib tiktoken openai

  Using cached contourpy-1.2.0-cp312-cp312-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/162.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/162.3 kB ? eta -:--:--
     -- ------------------------------------- 10.2/162.3 kB ? eta -:--:--
     ---------------------------- --------- 122.9/162.3 kB 1.2 MB/s eta 0:00:01
     -------------------------------------- 162.3/162.3 kB 1.2 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.5-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached pillow-10.2.0-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
    --------------------------------------- 0.2/7.6 MB 3.5 MB/s eta 0:00:03
   - -------------------------------------- 0.3/7.6 MB 3.9 MB/s eta 0:00:02
   -- ------------------------------------- 0

In [20]:
import pandas as pd
import numpy as np
import matplotlib

In [7]:
contents = []

for i in range(1, 8):
    file_name = f"./control_points/{i}.txt"
    try:
        with open(file_name, 'r', encoding="utf-8") as file:
            content = file.read()
            contents.append({'content': content})
    except FileNotFoundError:
        print(f"File {file_name} not found.")

df = pd.DataFrame(contents, columns=['content'])

In [9]:
import re

In [11]:
pd.options.mode.chained_assignment = None 
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['content']= df["content"].apply(lambda x : normalize_text(x))

In [13]:
import tiktoken

In [15]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["content"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]

7

In [24]:
import os
from openai import AzureOpenAI

In [25]:
os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['embedding'] = df["content"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [30]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(_df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    _df["similarities"] = _df["embedding"].apply(lambda x: cosine_similarity(x, embedding))

    res = (
        _df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

In [38]:
res = search_docs(df, "Le token de l'utilisateur se retrouve régulièrement invalidé dans un délai de moins de 180 jours. "
, top_n=4)

,content,n_tokens,embedding,similarities
6,Point de contrôle 13 ======= # Catégorie : Aut...,323,"[-0.00961134023964405, -0.00813981145620346, 0...",0.876890
5,Point de contrôle 23 ======= # Catégorie : Don...,384,"[-0.011622103862464428, 0.007696355693042278, ...",0.767541
0,Point de contrôle 1 ======= # Catégorie : Tier...,143,"[-0.017865363508462906, -0.019515523687005043,...",0.766913
3,Point de contrôle 4 ======= # Catégorie : Tier...,513,"[-0.027942441403865814, -0.019337281584739685,...",0.761505


In [41]:
system_prompt = """Objectif Principal : Le système doit permettre aux employés de l'ACPR d'accéder facilement aux documents normatifs relatifs à la supervision des activités des établissements financiers et de vérifier la conformité des phrases relatives aux produits financiers aux règlements en vigueur.
1. Recherche Documentaire :
•	Gérer les requêtes complexes en comprenant les relations entre les termes de recherche.
•	Si l’utilisateur ne parvient pas à trouver ce qu’il cherche, donne-lui des exemples de phrases à renseigner. Par exemple, donne-lui des mots clés ou des catégories. 
•	Si tu lis une abréviation, que ce soit par l’utilisateur ou dans un document, et que tu ne la comprends pas, dis que tu ne comprends l’abréviation et cite-la. 
2. Accès aux Informations :
•	Extraire de manière précise les informations pertinentes des documents normatifs en réponse aux requêtes de l'utilisateur.
•	Restituer le contexte entourant une information pour une meilleure compréhension. Faire bien attention, à séparer la citation du texte normatif des informations contextuelles. Le texte normatif doit être clairement identifiable et ne doit pas avoir été modifié. 
3. Conformité des Phrases aux Règlements :
•	Permettre à l'utilisateur de soumettre une phrase relative à un produit financier et demander si elle est conforme aux règlements en vigueur.
•	Analyser la phrase en utilisant les règles et les normes énoncées dans les documents normatifs.
4. Gestion des Erreurs et Ambiguïtés :
•	Gérer les situations où une requête est ambiguë ou incomplète en demandant des clarifications.
•	Si tu ne connais pas la réponse, dis-le et ne cherche pas à rajouter d’autres éléments à part des questions pour plus de précision.
5. Mises à Jour Légales :
•	Informer les utilisateurs des modifications récentes dans la législation financière.
6. Interaction Naturelle :
•	Favoriser une interaction conversationnelle naturelle avec l'utilisateur en comprenant le langage courant et en fournissant des réponses compréhensibles.
•	Réponds avec un langage formel est clair. Ce que tu écris doit pouvoir être présenté dans des rapports officiels. 
7. Assistance et Support :
•	Fournir un support contextuel pour aider les utilisateurs à formuler des requêtes de manière efficace. Attention, cela ne doit jamais modifier les textes normatifs dans tes réponses.

Points de contrôles sur lesquels baser la réponse : 

"""

In [51]:
os.environ["AZURE_OPENAI_DeploymentId"] = "gpt-35-turbo"

def generate_answer(prompt, system_prompt):
  res = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DeploymentId"),
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
  ).choices[0].message.content
  return res

def process_prompt(prompt):
  res = search_docs(df, prompt, top_n=2, to_print=False)
  sys_prompt = system_prompt + "\n".join(res["content"])
  answer = generate_answer(prompt, sys_prompt)
  return ({"answer": answer, "docs": res})

In [72]:
res = process_prompt("L'API a eu un niveau de disponibilité largement inférieur comparé à l'interface client sur septembre, est-ce conforme à la réglementation ?")

In [73]:
res["answer"]

"Selon l'article 67 de la DSP2, l'API doit fournir toutes les informations liées aux comptes de paiement sans discrimination par rapport à l'interface client. Si l'API a eu un niveau de disponibilité largement inférieur à celui de l'interface client, cela pourrait être considéré comme une non-conformité à la réglementation. Il est important de vérifier si cette situation est due à des limitations techniques ou à d'autres facteurs, et il est recommandé de contacter l'établissement concerné pour obtenir des explications et éventuellement demander les actions de remédiation nécessaires dans le respect des obligations réglementaires."

In [74]:
res["docs"]

,content,n_tokens,embedding,similarities
6,Point de contrôle 13 ======= # Catégorie : Aut...,323,"[-0.00961134023964405, -0.00813981145620346, 0...",0.804161
5,Point de contrôle 23 ======= # Catégorie : Don...,384,"[-0.011622103862464428, 0.007696355693042278, ...",0.803106
